# Ex 1.1 Motion Detection & Tracking
This task focuses on developing an application to detect and track moving cars from a camera recording. The algorithm of the application is based on the frame differencing and background subtraction techniques. The application will only focus on detecting cars that are in the "Main street".

In [8]:
# IMPORTING LIBRARIES FOR PROJECT
import cv2
import numpy as np

The traffic video ```Traffic_Laramie_1.mp4``` is loaded for verifying results.

In [9]:
# loading the video file
vid_path = "media/Traffic_Laramie_1.mp4"
capture = cv2.VideoCapture(vid_path)

# checking if video is opened successfully
if not capture.isOpened(): print("Error: Could not open video.")
else: print("Successfully opened video.")

Successfully opened video.


OpenCV will be used to implement the background subtraction in the algorithm.

This project will use the updated ```createBackgroundSubtractorMOG2()```
- ```history``` is the length of the history
- ```varThreshold``` is the threshold on the squared Mahalanobis distance between the pixel and the model to decide whether a pixel is well described by the background model
- ```detectShadows``` checks for shadows and marks them

The base values are used, and will be modified to better detect the cars.

In [10]:
background_sub = cv2.createBackgroundSubtractorMOG2( history = 500,
                                                     varThreshold = 16,
                                                     detectShadows = True )

Now, the algorithm will read the input video frame-by-frame, and apply background subtraction to extract the moving objects.